In [2]:
from dotenv import load_dotenv
import os
load_dotenv(override=True)

langsmith_api_key = os.getenv("LANGSMITH_API_KEY")

In [53]:
from langchain import hub as prompts
client = Client(api_key=langsmith_api_key)
prompt = client.pull_prompt("arduinomaster", include_model=True)

/opt/miniconda3/envs/mat/lib/python3.12/json/decoder.py:338: UserWarning: WARNING! extra_headers is not default parameter.
                extra_headers was transferred to model_kwargs.
                Please confirm that extra_headers is what you intended.
  obj, end = self.raw_decode(s, idx=_w(s, 0).end())


In [54]:
prompt

StructuredPrompt(input_variables=['language', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'arduinomaster', 'lc_hub_commit_hash': '9b1f4f1591626da67cbf6c2c7432b33a13b4826b450c76c87bc3da7ca6a5756e'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='You are ArduinoMaster, an expert electronics engineer and maker with 15+ years of experience building Arduino and Raspberry Pi projects. \n\n🔧 YOUR EXPERTISE:\n- Circuit design and troubleshooting\n- Arduino programming (C++) and Raspberry Pi (Python)\n- Sensor integration and hardware interfacing\n- PCB design and component selection\n- IoT project development\n\n⚡ YOUR RESPONSES SHOULD:\n- Provide clear, step-by-step instructions\n- Include specific pin numbers, resistor values, and component specs\n- Mention safety considerations and common pitfalls\n- Suggest code snippets when relevant\n- Recommend 

In [55]:
hydrated_prompt = prompt.invoke({
    "question": "How do I wire an ultrasonic sensor to Arduino for distance measurement?",
    "language": "English"
})
hydrated_prompt

{'answer': 'Short answer: Use an HC‑SR04 (or similar) VCC → 5V, GND → GND, TRIG → a digital output pin, ECHO → a digital input pin. Send a 10 µs pulse on TRIG, read the HIGH pulse width on ECHO (pulseIn), convert time to distance. Below are step‑by‑step wiring, a tested Arduino sketch, safety notes, and debugging tips.\n\nParts & components (recommendations)\n- Ultrasonic sensor: HC‑SR04 (common, cheap) or MaxBotix MB1010 / LV‑EZ series (higher quality)\n- Arduino Uno / Nano / Mega (5V logic) or an ESP32/3.3V board (see level shifting section)\n- 2 resistors for level shifting if using a 3.3V MCU: 10 kΩ and 5.1 kΩ (or 2 kΩ and 1 kΩ)\n- Jumper wires, breadboard, USB cable for power/programming\n\nWiring (for Arduino Uno / any 5V Arduino)\n1) HC‑SR04 VCC → Arduino 5V\n2) HC‑SR04 GND → Arduino GND\n3) HC‑SR04 TRIG → Arduino D9 (example; any digital pin)  \n4) HC‑SR04 ECHO → Arduino D10 (example; any digital input)  \n\nWiring colors (suggested): Red = 5V, Black = GND, Yellow = TRIG, Green

In [ ]:
from langsmith.client import convert_prompt_to_openai_format
from openai import OpenAI


openai_client = OpenAI()

converted = convert_prompt_to_openai_format(hydrated_prompt)

response = openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=converted["messages"],
)

print(response.choices[0].message.content)

In [51]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client

client=Client()

french_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your users can only speak French, make sure you only answer your users with French.

Conversation: {conversation}
Context: {context} 
Question: {question}
Answer:"""

french_prompt_template = ChatPromptTemplate.from_template(french_prompt)
client.push_prompt("french-rag-prompt", object=french_prompt_template)

'https://smith.langchain.com/prompts/french-rag-prompt/75567b82?organizationId=bcaa99f9-3a04-43e7-9c9d-c034ad1da1ea'

In [52]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client
from langchain_openai import ChatOpenAI

client=Client()
model = ChatOpenAI(model="gpt-4o-mini")

french_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your users can only speak French, make sure you only answer your users with French.

Conversation: {conversation}
Context: {context} 
Question: {question}
Answer:"""
french_prompt_template = ChatPromptTemplate.from_template(french_prompt)
chain = french_prompt_template | model
client.push_prompt("french-runnable-sequence", object=chain)

'https://smith.langchain.com/prompts/french-runnable-sequence/f1499fb1?organizationId=bcaa99f9-3a04-43e7-9c9d-c034ad1da1ea'